<a href="https://colab.research.google.com/github/ameliachoi/dacon-study-FUNDA/blob/master/dacon_funda_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DACON FUNDA : Predict Market Sales

## Index

1. Data load & Resampling
2. EDA
3. Modeling - Time Series
4. Modeling - Regression
5. Modeling - Deep Neural Network

## Data Field

- `store_id` : 상점 고유 아이디
- `card_id` : 사용한 카드의 고유 아이디
- `card_company` : 비식별화된 카드 회사
- `transacted_date` : 거래 날짜
- `transacted_time` : 거래 시간(시:분)
- `installment_term` : 할부 개월 수 (포인트 사용 시 (60개월 + 실제 할부개원)을 할부개월수에 기재한다)
- `region` : 상점의 지역
- `type_of_business` : 상점의 업종
- `amount` : 거래액(단위는 원이 아님)

---

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# basic setting
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from sklearn.metrics import make_scorer

# 시계열
from fbprophet import Prophet
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

# 회귀분석
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# deep neural network
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time

# setting
%matplotlib inline
pd.options.display.max_columns = 400
pd.options.display.float_format = '{:.5f}'.format 


### Evaluate Metric

- MAE(Mean Absolute Error) : 절대값 오차의 평균
- Validation을 위한 함수 생성 및 make scorer


In [5]:
def mae(prediction, correct):
  prediction = np.array(prediction)
  correct = np.array(correct)

  difference = correct - prediction
  abs_val = abs(difference)

  score = abs_val.mean()

  return score

mae_scorer = make_scorer(mae)
mae_scorer

make_scorer(mae)

### Step 1. Data Load & Resampling

#### Issue

1. 예측해야 하는 범위는 3개월인데 데이터는 시간 단위로 나뉘어져 있음

- 월 단위로 resampling 후, forecast 범위를 3개월로 지정해야 해결

2. 1967개의 `store_id`가 각각 trend와 seasonality를 가지고 있음

- 같은 알고리즘에서 각 `store_id`별로 parameter 조절

3. 예측 날짜는 2019-03 ~ 2019-05로 동일하나, 제공 데이터의 마지막 날짜는 차이가 존재함

- 마지막 날짜부터 3개월만 예측하다 제출

- 예측 기간이 길어질수록 오차가 크게 발생해 바로 뒤 3개월만 예측하는 것이 정확도가 높았음

In [7]:
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data09/funda_train.csv')
df_sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data09/submission.csv')

In [10]:
df_train.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.14286
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.14286
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.00000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.14286
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.00000


In [8]:
df_train.shape, df_sub.shape

((6556613, 9), (1967, 2))

In [9]:
df_train.columns

Index(['store_id', 'card_id', 'card_company', 'transacted_date',
       'transacted_time', 'installment_term', 'region', 'type_of_business',
       'amount'],
      dtype='object')

In [11]:
# datetime으로 데이터 타입 바꾸기
df_train['transacted_date'] = pd.to_datetime(df_train['transacted_date'])

In [12]:
df_train.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.14286
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.14286
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.00000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.14286
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.00000


In [15]:
frame_info = [(col, len(df_train[col].unique()), df_train[col].isnull().sum(), df_train[col].dtype, df_train[col].unique()[:5]) for col in df_train.columns]
df_info = pd.DataFrame(frame_info, columns=['name', 'num_of_unique', 'num_of_nan', 'type', 'front5_values'])

In [16]:
df_info

,name,num_of_unique,num_of_nan,type,front5_values
0,store_id,1967,0,int64,"[0, 1, 2, 4, 5]"
1,card_id,3950001,0,int64,"[0, 1, 2, 3, 4]"
2,card_company,8,0,object,"[b, h, c, a, f]"
3,transacted_date,1003,0,datetime64[ns],"[2016-06-01T00:00:00.000000000, 2016-06-02T00:..."
4,transacted_time,1440,0,object,"[13:13, 18:12, 18:52, 20:22, 11:06]"
5,installment_term,34,0,int64,"[0, 2, 3, 60, 4]"
6,region,181,2042766,object,"[nan, 서울 종로구, 충북 충주시, 부산 동래구, 경기 평택시]"
7,type_of_business,146,3952609,object,"[기타 미용업, nan, 의복 액세서리 및 모조 장신구 도매업, 한식 음식점업, 배..."
8,amount,30551,0,float64,"[1857.142857142857, 857.1428571428571, 2000.0,..."


#### 시계열 분석을 위해 date 정보 index로 변환

In [17]:
df_train = df_train.set_index('transacted_date')
df_train.head(3)

,store_id,card_id,card_company,transacted_time,installment_term,region,type_of_business,amount
transacted_date,,,,,,,,
2016-06-01,0,0,b,13:13,0,NaN,기타 미용업,1857.14286
2016-06-01,0,1,h,18:12,0,NaN,기타 미용업,857.14286
2016-06-01,0,2,c,18:52,0,NaN,기타 미용업,2000.00000


#### 시간 단위로 나뉘어져 있는 데이터 일단위로 resampling

- `store_id` / `region` / `type_of_business` :  기존과 동일
- `day_of_week` : 각 요일을 나타내는 숫자. 월요일은 0, 일요일은 6.
- `business_day` : working day 여부. 1이면 working day.
- `num_of_pay` : 일 결제 건수. `card_id`의 count로 생성
- `num_of_revisit` : 단골 방문 횟수. `card_id`의 value 중 count >= 3 value의 결제 건수
- `installment_term` : 일 총 할부 개월 수, 기존 `installment_term`의 합. (수정)
- `amount` : 일 매출액. 기존 amount의 합. (수정)

In [21]:
def resample_day(train_df):
  df_day = pd.DataFrame()

  for i in train_df['store_id'].unique():
    df_num = train_df[train_df['store_id']==i] # store_id 횟수 count

    # 'card_id'의 일별 counting을 통해 일 거래 횟수 확인
    # resample 함수의 rule='d'인 경우 일별 그룹이 가능해지고, count함수를 통해 unique값별로 빈도수 생성
    count_cols = df_num['card_id'].resample(rule='d').count().rename('num_of_pay')

    # 'card_id' value count >= 3이면 단골로 판단하고 단골 방문 횟수 확인
    revisit_idx = df_num['card_id'].value_counts().reset_index().query('card_id > 2')['index'].values
    revisit_ct = df_num[df_num['card_id'].isin(revisit_idx)]['card_id'].resample(rule='d').count().rename('num_of_revisit')

    # 할부 개월수와 매출액은 일단위로 합
    sum_cols = df_num[['installment_term', 'amount']].resample(rule='d').sum()
    df_num_day = pd.concat([count_cols, revisit_ct, sum_cols], axis=1)

    # 매장 id, 지역, 업종 분류 추가
    df_num_day.insert(0, 'store_id', i)
    df_num_day.insert(4, 'region', df_num[df_num['store_id']==i]['region'].unique()[0])
    df_num_day.insert(5, 'type_of_business', df_num[df_num['store_id']==i]['type_of_business'].unique()[0])

    df_day = pd.concat([df_day, df_num_day], axis=0)

  df_day.insert(1, 'day_of_week', df_day.index.dayofweek) #df_day가 일별로 묶여 index에 들어가있음
  df_day.insert(2, 'business_day', df_day['day_of_week'].replace({0:1, 2:1, 3:1, 4:1, 5:0, 6:0}).values)
  df_day['num_of_revisit'].fillna(0, inplace=True)

  return df_day

In [22]:
%%time
df_day = resample_day(df_train)

CPU times: user 3min 29s, sys: 4.86 s, total: 3min 33s
Wall time: 3min 34s


In [23]:
df_day.head()

,store_id,day_of_week,business_day,num_of_pay,num_of_revisit,installment_term,region,type_of_business,amount
transacted_date,,,,,,,,,
2016-06-01,0,2,1,4,4.00000,0,NaN,기타 미용업,12571.42857
2016-06-02,0,3,1,7,3.00000,0,NaN,기타 미용업,40571.42857
2016-06-03,0,4,1,3,2.00000,0,NaN,기타 미용업,18142.85714
2016-06-04,0,5,0,7,3.00000,0,NaN,기타 미용업,31714.28571
2016-06-05,0,6,0,3,3.00000,0,NaN,기타 미용업,10428.57143


In [24]:
df_day.isnull().sum()

store_id                  0
day_of_week               0
business_day              0
num_of_pay                0
num_of_revisit            0
installment_term          0
region               629927
type_of_business    1087791
amount                    0
dtype: int64

In [25]:
df_day.to_csv('preprocessed.csv')